In [2]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [7]:
class Dictionary(object):
    def __init__(self) -> None:
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word) -> None:
        if word not in self.word2idx:        
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self) -> int:
        return len(self.word2idx)

In [15]:
class TextProcess(object):

    def __init__(self) -> None:
        self.dictionary = Dictionary()
    
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        rep_tensor = torch.LongTensor(tokens) # tensor of length tokens
        index = 0

        # Capture the indices of each word in the tensor for the entire story
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        
        # Divide the data into batches based on batch size 
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches*batch_size]
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor


In [19]:
embed_size = 128         # Input features to LSTM. idx -> word embedding size
hidden_size = 1024       # Num of hidden neurons or num of hidden units
num_layers = 1           # 1 layer of LSTM. If 2, 2 LSTM layers stacked on top of each other
num_epochs = 20
batch_size = 20
timesteps = 30           # we look at 30 previous words to predict next word
learning_rate = 0.002    # we can use LR scheduler here

In [20]:
corpus = TextProcess()
rep_tensor = corpus.get_data('17.alice.txt', batch_size)
print(rep_tensor.shape)

torch.Size([20, 1484])


In [27]:
vocab_size = len(corpus.dictionary)
print(vocab)

5290


In [22]:
# batches based on time steps
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [25]:
class TextGenerator(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) # performs the mapping from idx to word embedding
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h) # x -> input, h -> initialzed hidden state and memory state
        # Reshape (batch_size, timesteps, embed_size) -> (batch_size*timesteps, hidden_size) to be appropriate for FC layer
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [28]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in num_epochs:

    # set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    